In [1]:
from src.nws import NWSClient, celsius_to_fahrenheit, fahrenheit_to_celsius
from src.params import *
from datetime import datetime, timedelta
import pytz

client = NWSClient()

In [28]:
station = StationID.MIA
radii = {
    StationID.MDW: 4,
    StationID.MIA: 4,
}
station_tz = STATION_TZ[station]
prior_days = 1

dfs = client.get_timeseries(station, datetime.now(tz=BASE_TZ) - timedelta(days=prior_days), datetime.now(tz=BASE_TZ), radius=radii[station])

for df in dfs:
    df.rename(columns={
        'air_temp_set_1': 'temperature', 
        'relative_humidity_set_1': 'relative_humidity',
        'wind_speed_set_1': 'wind_speed',
        'wind_direction_set_1': 'wind_direction',
        'air_temp_high_6_hour_set_1': 'temperature_high_6_hour',
        'air_temp_high_24_hour_set_1': 'temperature_high_24_hour',
    }, inplace=True)

    df['temperature_f'] = celsius_to_fahrenheit(df['temperature'])

[df.info() for df in dfs]


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 313 entries, 2024-11-12 23:00:00-05:00 to 2024-11-13 22:55:00-05:00
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   temperature               313 non-null    float64
 1   relative_humidity         313 non-null    float64
 2   wind_speed                313 non-null    float64
 3   wind_direction            313 non-null    float64
 4   temperature_high_24_hour  1 non-null      float64
 5   temperature_high_6_hour   4 non-null      float64
 6   time                      313 non-null    object 
 7   station_id                313 non-null    object 
 8   station_name              313 non-null    object 
 9   distance                  313 non-null    float64
 10  temperature_f             313 non-null    float64
dtypes: float64(8), object(3)
memory usage: 29.3+ KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1 entries, 2024-11-13 07:

[None, None, None, None, None]

In [ ]:
import time
from IPython.display import clear_output
# MIA - 11:07 AM CT
# MDW - 11:08 AM CT - still too uncertain
# AUS - 11:23 AM CT - still too uncertain
# NYC 

while True:
    print(f"Query Time: {time.ctime()}")

    dfs = client.get_timeseries(station, datetime.now(tz=BASE_TZ) - timedelta(days=prior_days), datetime.now(tz=BASE_TZ), radius=radii[station])
    df = df.rename(columns={
            'air_temp_set_1': 'temperature', 
            'relative_humidity_set_1': 'relative_humidity',
            'wind_speed_set_1': 'wind_speed',
            'wind_direction_set_1': 'wind_direction',
            'air_temp_high_6_hour_set_1': 'temperature_high_6_hour',
            'air_temp_high_24_hour_set_1': 'temperature_high_24_hour',

        })
    df[['temperature', 'temperature_high_6_hour', 'temperature_high_24_hour']] = df[['temperature', 'temperature_high_6_hour', 'temperature_high_24_hour']].apply(celsius_to_fahrenheit)

    # df.info()
    display(df.tail(25))
    clear_output(wait=True)
    time.sleep(15)

In [29]:
import pandas as pd
from functools import reduce

res_df = pd.concat([df.reset_index() for df in dfs], axis=0, ignore_index=True)
res_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype                           
---  ------                    --------------  -----                           
 0   date_time                 720 non-null    datetime64[ns, America/New_York]
 1   temperature               720 non-null    float64                         
 2   relative_humidity         602 non-null    float64                         
 3   wind_speed                602 non-null    float64                         
 4   wind_direction            518 non-null    float64                         
 5   temperature_high_24_hour  2 non-null      float64                         
 6   temperature_high_6_hour   4 non-null      float64                         
 7   time                      720 non-null    object                          
 8   station_id                720 non-null    object                          
 9   station_na

In [30]:
import pandas as pd
pd.options.plotting.backend = "plotly"

import plotly.express as px

fig = px.line(res_df, x="date_time", y="temperature_f", color="station_name", title=f"Temperature (°F) for Station {station}", width=1800, height=900)
fig.update_traces(mode="lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.update_yaxes(title="Temperature (°F)")
fig.update_xaxes(title="Date")

fig.show()